In [1]:
import numpy as np
#from sympy.abc import x,y,z
from sympy import ordered, Matrix, hessian, lambdify, diff, latex
from sympy.vector import CoordSys3D, gradient
from misc_tools.print_latex import print_tex

input example : 
>>> arr_T = np.array([[r'\vec{v}_1', r'\vec{v}_2']]).T
>>> print_tex(arr_T,'=', np.arange(1,5).reshape(2,-1)/4, r'; symbols: \otimes, \cdot,\times')
output: 


<IPython.core.display.Math object>

## Comments on sympy
I am trying to use 'CoordSys3D' class within vector module, because it has properly defined operators. 

For example gradient has to be written manually for arbitrary or we can use jacobian to compute it (see in the end of notes)

In [2]:
C = CoordSys3D('C')
x,y,z = C.x,C.y,C.z
f_x = 2*x*y**3 
f_y = 3*y**2*z
f_z = z**3*x
display(f_x , f_y , f_z)

2*C.x*C.y**3

3*C.y**2*C.z

C.x*C.z**3

***
## Derivative of a vector function
Vector function is a vector with each component being a function of some independent variables (i.e $x,y,z$):
$$\vec{f}(x,y) = \begin{bmatrix} 
f_x(x,y) \\ f_y(x,y)
\end{bmatrix}= \begin{bmatrix} 
f_x(x,y,z) & f_y(x,y,z)
\end{bmatrix}^T$$
We can rewrite vector function in terms of basis ($\vec{e}_x = \hat{i},\dots$)explicitly:
$$\vec{f}(x,y) = f_x(x,y,z) \hat{i}+  f_y(x,y,z) \hat{j}$$
If we decide to take a derivative of a vector function, we can use linearity of derivative and apply it to each term in a sum
$$\frac{\partial \vec{f}}{\partial x} = \frac{\partial f_x}{\partial x}\hat{i} + \frac{\partial f_y}{\partial x}\hat{j} = \begin{bmatrix} 
\frac{\partial f_x}{\partial x}& \frac{\partial f_y}{\partial x}
\end{bmatrix}^T$$ 

Takeaway: derivative operator acts on each element of vector function without modifying its shape.

(Optional):
> One can write that:
>$$\frac{\partial \vec{f}}{\partial x_i} = \sum_j \frac{\partial f_j}{\partial x_i} \vec{e}_j$$  
>or $j$-th component is 
>$$\bigg(\frac{\partial \vec{f}}{\partial x_i}\bigg)_j = \frac{\partial f_j}{\partial x_i}$$

In [3]:
f_v = Matrix([f_x,f_y,f_z])
df_v_dx = f_v.diff(x)
print_tex(r'\frac{\partial \vec{f}}{\partial x}=',r'\frac{\partial}{\partial x}', np.array(f_v), '=',np.array(df_v_dx))


<IPython.core.display.Math object>

***
## Gradient of a function
Gradient $\nabla \cdot$ of a scalar function shows the direction of steepest increase in value of the function
$$\nabla f(x_1,x_2) = \nabla f(x,y) = 
\begin{bmatrix} 
\frac{\partial f}{\partial x} (x,y) \\ \frac{\partial f}{\partial y} (x,y)
\end{bmatrix}
$$
One can think of $\nabla \cdot$ derivative vector operator
$$\nabla  = \frac{\partial }{\partial x}\hat{i}  + \frac{\partial }{\partial y}\hat{j}=  \begin{bmatrix} 
\frac{\partial }{\partial x} \\ \frac{\partial }{\partial y}
\end{bmatrix}=
\begin{bmatrix} 
\frac{\partial }{\partial x} & \frac{\partial }{\partial y}
\end{bmatrix}^T
$$
which acts on a scalar function from the left.

Takeaway: similarly to vector-scalar multiplication, gradient produces a vector of same shape.

In [4]:
f_xyz = f_x + f_y + f_z
print_tex('f(x,y,z) = ', latex(f_xyz))
grad = gradient(f_xyz)
print_tex(r'\sum_i (\nabla f)_i \vec{e}_i =',latex(grad))
grad_true = Matrix([[grad.dot(basis_vec) for basis_vec in [C.i, C.j, C.k]]]).T # extract components
print_tex(r'\nabla f = ', np.array(grad_true))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

***
## Jacobian matrix

Jacobian matrix of a vector function is a storage of different derivatives of a vector:
$$ \vec{J}(\vec{f}(x,y)) = \begin{bmatrix} 
\frac{\partial \vec{f}}{\partial x} & \frac{\partial \vec{f}}{\partial y}
\end{bmatrix}$$
Since we have seen that
$$\frac{\partial \vec{f}}{\partial x} =  \frac{\partial}{\partial x} \begin{bmatrix} 
f_x \\ f_y
\end{bmatrix} = 
\begin{bmatrix} 
\frac{\partial f_x}{\partial x} \\ \frac{\partial f_y}{\partial x}
\end{bmatrix}$$
Then jacobian is $n \times n$ matrix, where $n = |x,y,\dots|$

$$\vec{J}(\vec{f}(x,y)) = \begin{bmatrix} 
\frac{\partial f_x}{\partial x} & \frac{\partial f_x}{\partial y} \\
\frac{\partial f_y}{\partial x} & \frac{\partial f_y}{\partial y}
\end{bmatrix}$$

>Naturally following also applies here
>$$\bigg[\vec{J}(\vec{f})\bigg]_{i,j} = \frac{\partial f_j}{\partial x_i}$$

In [5]:
f_v_xy = Matrix([[f_x,  f_y]]).T
jac_f = f_v_xy.jacobian([C.x, C.y])
print_tex(r'\vec{J}(\vec{f}_{xy})=\vec{J}(',np.array(f_v_xy),')=', np.array(jac_f))


<IPython.core.display.Math object>

>Jacobian can be used to compute a gradient of a function. For it we need only one component vector:
>$$\nabla f = \bigg\{\vec{J}(\vec{f})\bigg\}_{i=0,j\in[x,y,z]}^T$$
>It is important if we dont use pre-defined coordinate system

In [6]:
grad_v_jac      = Matrix([f_xyz]).jacobian([x,y,z]).T
print_tex(r'\nabla_{true} = ', np.array(grad_true), r';\nabla_{jac} = ', np.array(grad_v_jac))

<IPython.core.display.Math object>

***
## Hessian
One may choose to take a jacobian of gradient of scalar function
$$ \vec{J}(\nabla f) = \begin{bmatrix} 
\frac{\partial }{\partial x}\nabla f & \frac{\partial}{\partial y}\nabla f
\end{bmatrix}
=
\begin{bmatrix} 
\frac{\partial}{\partial x}\begin{bmatrix} \frac{\partial f}{\partial x} \\ \frac{\partial f}{\partial y}\end{bmatrix}
 & \frac{\partial}{\partial y} \begin{bmatrix} \frac{\partial f}{\partial x} \\ \frac{\partial f}{\partial y} \end{bmatrix}
\end{bmatrix}=
\begin{bmatrix} 
\frac{\partial}{\partial x} \frac{\partial f}{\partial x} & \frac{\partial}{\partial y} \frac{\partial f}{\partial x} \\
\frac{\partial}{\partial x} \frac{\partial f}{\partial y} & \frac{\partial}{\partial y} \frac{\partial f}{\partial y} \\
\end{bmatrix}=
\underbrace{
    \begin{bmatrix} 
\frac{\partial}{\partial x} \frac{\partial }{\partial x} & \frac{\partial}{\partial y} \frac{\partial }{\partial x} \\
\frac{\partial}{\partial x} \frac{\partial }{\partial y} & \frac{\partial}{\partial y} \frac{\partial }{\partial y} \\
\end{bmatrix}
}_{H} f
$$

As we can see that $\vec{J}(\nabla \cdot) = H (\cdot)$ where $H$ is Hessian matrix (operator)

Entry $i,j$ for Hessian is
$$H_{j,i} = \frac{\partial}{\partial x_i \partial x_j} = \frac{\partial}{\partial x_j \partial x_i} = H_{i,j} $$
Due to symmetry of second derivatives
$$H_{2,1} = H_{1,2} = \frac{\partial}{\partial x_2 \partial x_1} =  \frac{\partial}{\partial y \partial x} = \frac{\partial}{\partial x \partial y}$$


In [7]:
hess = hessian(f_xyz,[x,y,z])
print_tex('H(f)=H(',latex(f_xyz),') = ', np.array(hess))
hess_v2 = grad_true.jacobian([x,y,z])
print_tex(r'\vec{J}(\nabla f)=',np.array(hess_v2))

<IPython.core.display.Math object>

<IPython.core.display.Math object>